In [1]:
import pandas as pd

## The extracting stage

The extracting stage is happening in "scrape_row_jobs.py".

In this stage I web-scraped data from search results of Indeed.com. To get the HTML of the pages I used requests Python library and for scraping I used bs4 Python library.

I decided that it would be easier to get required experience levels for jobs by using Indeed filters rather than scraping this information from jobs' descriptions.

In order to avoid receiving a CAPTCHA I added few time.sleep() functions for a few seconds to imitate human browsing speed.

You can see the result of this stage as the dataset below.

In [2]:
raw_jobs = pd.read_pickle("data/raw_jobs_copy.pkl")
raw_jobs.head()

,url,job_info,experience_level
0,https://www.indeed.com/viewjob?jk=00a4d7932929...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Entry level
1,https://www.indeed.com/viewjob?jk=0101872c28a8...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Entry level
2,https://www.indeed.com/viewjob?jk=011d3dd5eb0c...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Senior level
3,https://www.indeed.com/viewjob?jk=011d508f79b1...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Senior level
4,https://www.indeed.com/viewjob?jk=013d358f1a88...,"<div class=""jobsearch-ViewJobLayout-jobDisplay...",Mid level


## The transforming stage

The transforming stage is happening in "extract_jobs_features.py".

In this stage I extracted different features from the jobs loaded previously (you can see all of them in the dataset shown below). For these purposes I used bs4, re and nltk Python libraries.

In [3]:
jobs = pd.read_pickle("data/jobs.pkl")
jobs.head()

,url,job_title,company_name,company_rating,number_of_reviews,annual_salary,location,remote,full_time,temporary,internship,experience_level,degree,requirements
0,https://www.indeed.com/viewjob?cmp=lcoleman%40...,Data Scientist I,None,None,None,59600,"Fort Benning, GA 31905",False,True,False,False,Entry level,Bachelor,"statistical software,microsoft office,programm..."
1,https://www.indeed.com/viewjob?jk=001582624bed...,Product Review Clerk,None,3.2,434,44400,"Lemoore, CA 93245",False,True,False,False,Mid level,Bachelor,"microsoft office,excel"
2,https://www.indeed.com/viewjob?jk=00432e5b8ab8...,Senior Data Analyst,Sysco Labs,4.1,9,97700,"Austin, TX 78702",False,True,False,False,Senior level,Bachelor,"data visualization,power bi,cloud technologies..."
3,https://www.indeed.com/viewjob?jk=0044911b6206...,Data Analyst - Workplace Experience,Electronic Arts,3.8,555,None,"Los Angeles, CA",False,True,False,False,Mid level,Bachelor,"sql,tableau,power bi,data visualization"
4,https://www.indeed.com/viewjob?jk=00486361f7b8...,"Senior Deep Learning Scientist, Prediction - A...",None,4.2,248,None,"Redmond, WA",False,True,False,False,Senior level,PhD,"tensorflow,machine learning with python,progra..."


The second part of the stage is happening in "transform_requirements_data.py".

In this part I used jobs feature to get dataset that gives more convinient access to requirements data. The dataset is a table that keeps track of requirements' occurrences based on filters such as experience_level, degree, remote, etc.

For example, row 3 tells us that we have found 5 remote, full-time, permanent jobs offers that require entry level experience, no degree, SQL knowledge and that is not an internship.

In [4]:
reqs_df = pd.read_pickle("data/requirements.pkl")
reqs_df.head()

,group_of_requirements,requirement,experience_level,degree,remote,full_time,temporary,internship,number_of_occurrences
0,sql,sql,Entry level,No degree,True,True,True,True,0
1,sql,sql,Entry level,No degree,True,True,True,False,0
2,sql,sql,Entry level,No degree,True,True,False,True,0
3,sql,sql,Entry level,No degree,True,True,False,False,29
4,sql,sql,Entry level,No degree,True,False,True,True,0


## The loading stage

The loading stage is happening in "load_raw_jobs_to_db.py" and "load_features_to_db.py".

In this stage I connected to AWS RDS database using pymysql Python library, created table and loaded the raw_jobs and jobs datasets using MySQL to the database.

Below you can see the tables schemas.

CREATE TABLE IF NOT EXISTS raw_jobs (
       url VARCHAR(255) NOT NULL PRIMARY KEY,
       job_info VARCHAR(15000),
       experience_level VARCHAR(255)
)

CREATE TABLE IF NOT EXISTS jobs (
    url VARCHAR(255) NOT NULL PRIMARY KEY,
    job_title VARCHAR(255),
    company_name VARCHAR(255),
    company_rating INT,
    number_of_reviews INT,
    annual_salary FLOAT,
    location VARCHAR(255),
    remote BOOLEAN,
    full_time BOOLEAN,
    temporary BOOLEAN,
    internship BOOLEAN,
    experience_level VARCHAR(255),
    degree VARCHAR(255),
    requirements VARCHAR(1024)
)

## The analysis stage

The analysis stage is happening in "analysis_and_visualizations.ipynb".

In this stage I built visualizations of the collected data to get valuable insights about the field of data science.